<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/what_is_lsdtopotools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is lsdtopotools?

Last updated by Simon M Mudd on 11/05/2023

`lsdtopotools` is a collection of topographic processing routines. It has been developed and used for scientific research; you can cite the software via its Zenodo record, which has a doi: https://zenodo.org/record/7892465#.ZFziJBHMK70

You can install `lsdtopotools` using the package management software `conda` or `mamba`. To run a topographic analysis, you call one of the `lsdtopotools` programs. You can run these from a command line but we also include a python wrapper. 

We also have `lsdviztools`, a python package used for plotting `lsdtopotools` output. `lsdviztools` contains the python wrappers for `lsdtopotools`. The objective of `lsdviztools` is to produce pretty plots that you can use in presentations and publications, it is not meant to be a general geospatial visualisation package.  

In this notebook I will install `lsdtopotools` in google colab and give some information about file formats. You can also run this notebook using the docker container for lsdtopotools: https://hub.docker.com/r/lsdtopotools/lsdtt_pytools_docker If you don't know what docker is, just ignore that part and start with the colab instructions. 

## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **Download some data** section. 

First we install `lsdviztools`. This will take around a minute. It is important you do this before the `condacolab` step. 

In [ ]:
!pip install lsdviztools &> /dev/null

Now we need to install lsdtopotools. We do this using something called `mamba`. To get `mamba` we install something called `condacolab`. 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Now use mamba to install `lsdtopotools`. 
This step takes a bit over a minute. 

In [ ]:
!mamba install -y lsdtopotools &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. This notebook was tested on version 0.8.

In [ ]:
!lsdtt-basic-metrics -v

## Download some data for the examples

Before we go on to some examples we need to get a bit of data (a 4 Mb DEM from the Shillong Plateau that we analysed in Strong et al, 2019, https://doi.org/10.1016/j.geomorph.2019.01.016):

In [ ]:
# Get the data from github
import urllib
urllib.request.urlretrieve("https://raw.githubusercontent.com/LSDtopotools/ExampleTopoDatasets/master/ChiAnalysisData/Meghalaya/Mega_divide.hdr", "Mega_divide.hdr")
urllib.request.urlretrieve("https://raw.githubusercontent.com/LSDtopotools/ExampleTopoDatasets/master/ChiAnalysisData/Meghalaya/Mega_divide.bil", "Mega_divide.bil")

## The `lsdtopotools` functions

Most users will run `lsdtopotools` by using out python wrapper, but it is worth explaining how the tools work in the first instance. 

In any python notebook, you can access the underlying linux shell (yes there is linux under here) by using the `!` symbol (pronounced "bang"). 

A basic linux command is to list files using the `ls` command, so here we can do this in the notebook. 

In [ ]:
!ls -l

`lsdtopotools` is not python, it is actually written in c++ and you call it with some programs that you get via the linux command line using the `!` command. 


In [ ]:
!lsdtt-basic-metrics

So here you see a message that says "FATAL ERROR". It is because `lsdtopotools` expects a file with instructions. We call these either parameter files or driver files. 

### The basic ingredients of an `lsdtopotools` parameter file

In a parameter file you need to tell it what DEM you are working on with the keywords `read fname`. You can also designate the files to be written with `write fname`. The program assumes you are reading and writing to the current directory (you can change the directory but we won't bother with that here). 

For any parameter in an `lsdtopotools` analysis you need a colon (`:`) after the keyword. 

The program won't do anything if you only give it the name of the file to read. If we want to make if do some analysis, you need to tell it what to do. 

A basic thing you might want is to write a hillshade file, which can be done by setting the `write_hillshade` parameter to `true`. 

Let's write a very basic driver file now:

In [ ]:
# Write the driver file. Each instruction is on its own line. 
with open('mega_01.driver', 'w') as file:
  file.write("# Any line starting with a # is a comment and is ignored by lsdtopotools\n")
  file.write("read fname: Mega_divide\n")
  file.write("write fname: Mega_divide\n")
  file.write("write_hillshade: true\n")

In [ ]:
# We look at the file to make sure it is correct. 
!cat mega_01.driver

### Running an analysis with one of the `lsdtopotools` programs

Okay, now that we have a parameter file, we can run one of the `lsdtopotools` programs. There are six programs:

* `lsdtt-basic-metrics`
* `lsdtt-channel-extraction`
* `lsdtt-cosmo-tool`
* `lsdtt-valley-metrics`
* `lsdtt-chi-mapping`
* `lsdtt-hillslope-channel-coupling`

It turns out the driver file we just made (`mega_01.driver`) will do the same thing if we use it with any of the programs, because all of them have the ability to write a hillshade. But for this example we will use `lsdtt-basic-metrics`:

In [ ]:
!lsdtt-basic-metrics mega_01.driver

Right, if you are in google colab, you can look at the files (click on the folder to your left) but we can also use the `ls` command:

In [ ]:
!ls

So that analysis made two new files `Mega_divide_hs.bil` and `Mega_divide_hs.hdr`. 
You get two files since all `lsdtopotools` processing used ENVI bil format and such files have a header file. 
The reason we don't use geotiff is because you need a bunch of libraries for that in c++ and we try to keep the dependencies of the source code to a minimum. See here about converting from different formats to ENVI bil: https://lsdtopotools.github.io/LSDTT_documentation/LSDTT_introduction_to_geospatial_data.html#_gdal 

Any hillshade generated by `lsdtopotools` will have `_hs` in the filename. 

We can look at the resulting raster using `matplotlib` and `rasterio`

In [ ]:
# Plot the hillshade. We need to set the colour max and minimum since imshow won't deal with the raster nodata.
import rasterio
from matplotlib import pyplot
src = rasterio.open("Mega_divide_hs.bil")
pyplot.imshow(src.read(1), cmap='gray',vmin = 0,vmax = 265)

Hey, fantasic, we have a hillshde. What units are those? Well, they are in pixels. So this isn't georeferenced. 

We can use `lsdviztools` to show the hillshade however. `lsdviztools` likes to drape semitransparent rasters over the hillshade, because it looks nice. In the default simple hillshade function below it uses the elevation raster for this:

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
Base_file = "Mega_divide"
DataDirectory = "./"
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology",dpi=500)

## Getting all the analysis options from one of the `lsdtopotools` programs

To get the different options for analyses available in each program, you can call the help function. This prints both a csv and html file, each with all the available parameters and some explanation. 

Let's look at `lsdtt-basic-metrics`

In [ ]:
!lsdtt-basic-metrics -h

We can load the csv file and then look at all the options. 

In general the data type of the key analyses are boolean, so we can have a look at all the boolean options to get a sense of what is possible in `lsdtt-basic-metrics`

In [ ]:
import pandas as pd
bm_df = pd.read_csv("lsdtt-basic-metrics-README.csv", on_bad_lines='skip')
bm_df.head()

Now we can look at the first twenty boolean options

In [ ]:
bool_df = bm_df.loc[bm_df.type == "bool"]
bool_df.head(20)

## Using the `lsdtopotools` python wrapper

If you don't like using the command line and writing your own files, `lsdviztools` has a python wrapper for that. 

Here is an example (I'm getting d8 drainage area, dinf drainage area, and slope):

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw
lsdtt_parameters = {"write_hillshade" : "true",
                    "print_d8_drainage_area_raster" : "true",
                    "print_dinf_drainage_area_raster" : "true",
                    "print_slope" : "true"}
r_prefix = "Mega_divide"
w_prefix = "Mega_divide"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

If you look at the pattern above, you should see that you add parameters using a dictionary structre, and you can control the read and write prefixes, and the directories to which you write the data. You can control which program you use. 

One note is that the parameters must always be fed a strings (so even if the the parameter is an integer or float, you still add it to the dictionary as a string (i.e., in quotation marks). 

This python wrapper write a driver file and calls the program, just like before. 

I'll just give a few examples below of looking at the data with `lsdviztools`

In [ ]:
### Plot the d-inf area  
Base_file = "Mega_divide"
r_prefix = "Mega_divide"
Drape_prefix = r_prefix+"_dinf_area"
DataDirectory = "./"
this_area_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="pink", 
                                       cbar_label = "Drainage area",
                                       cbar_loc = "bottom",
                                       drape_cnorm = "LogNorm",
                                       colour_min_max = [1000,10000000],
                                       save_fig=True, size_format="ESURF",dpi=600)

Hey? Where is my image??? Because we used save_fig=True, the image is not displayed but rather printed to file. So we can look at it with a call to display:

In [ ]:
print(this_area_img)
from IPython.display import display, Image
display(Image(filename=this_area_img, width=600))

In [ ]:
### Plot the d8  
Base_file = "Mega_divide"
r_prefix = "Mega_divide"
Drape_prefix = r_prefix+"_d8_area"
DataDirectory = "./"
this_area_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="pink", 
                                       cbar_label = "Drainage area",
                                       cbar_loc = "bottom",
                                       drape_cnorm = "LogNorm",
                                       colour_min_max = [1000,10000000],
                                       save_fig=True, size_format="ESURF",dpi=600)

In [ ]:
print(this_area_img)
from IPython.display import display, Image
display(Image(filename=this_area_img, width=900))

In [ ]:
### Plot the gradient 
Base_file = "Mega_divide"
r_prefix = "Mega_divide"
Drape_prefix = r_prefix+"_SLOPE"
DataDirectory = "./"
this_area_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="coolwarm", 
                                       cbar_label = "gradient (m/m)",
                                       cbar_loc = "bottom",
                                       colour_min_max = [0,1],
                                       save_fig=True, size_format="ESURF",dpi=600)

In [ ]:
print(this_area_img)
from IPython.display import display, Image
display(Image(filename=this_area_img, width=900))

Okay, hopefully this has given you an idea about the structure of `lsdtopotools` and `lsdviztools`, and you can use the other notebooks in this repository to do a wider variety of analyses. 